In [1]:
# geosam
import os
import leafmap
from samgeo import SamGeo

import torch

# image conversion
import rasterio
import numpy as np
from skimage import img_as_ubyte


In [2]:
m = leafmap.Map(center=[49.8653, -99.981], zoom=20, height="500px")
m.add_basemap("SATELLITE")
# m

In [3]:
# draw polygon to set m.user_roi

bbox=m.user_roi_bounds()
if bbox is None:
    bbox = [-99.9815, 49.8653, -99.9803, 49.8654]
print(bbox)

[-99.9815, 49.8653, -99.9803, 49.8654]


In [4]:
## initialize sam

device = 'cuda' if torch.cuda.is_available() else 'cpu'
sam = SamGeo(
    model_type='vit_h', # can be vit_h 2.4G, vit_l 1.2G, vit_b 358M
    device=device,
    automatic=False,
    sam_kwargs=None,
)

In [6]:
"""
kwargs
 sam_kwargs (dict, optional): Optional arguments for fine-tuning the SAM model. Defaults to None.
                The available arguments with default values are listed below. See https://bit.ly/410RV0v for more details.

                points_per_side: Optional[int] = 32,
                points_per_batch: int = 64,
                pred_iou_thresh: float = 0.88,
                stability_score_thresh: float = 0.95,
                stability_score_offset: float = 1.0,
                box_nms_thresh: float = 0.7,
                crop_n_layers: int = 0,
                crop_nms_thresh: float = 0.7,
                crop_overlap_ratio: float = 512 / 1500,
                crop_n_points_downscale_factor: int = 1,
                point_grids: Optional[List[np.ndarray]] = None,
                min_mask_region_area: int = 0,
                output_mode: str = "binary_mask",

        Using a SAM model, generates masks for the entire image.
        Generates a grid of point prompts over the image, then filters
        low quality and duplicate masks. The default settings are chosen
        for SAM with a ViT-H backbone.

        Arguments:
          model (Sam): The SAM model to use for mask prediction.
          points_per_side (int or None): The number of points to be sampled
            along one side of the image. The total number of points is
            points_per_side**2. If None, 'point_grids' must provide explicit
            point sampling.
          points_per_batch (int): Sets the number of points run simultaneously
            by the model. Higher numbers may be faster but use more GPU memory.
          pred_iou_thresh (float): A filtering threshold in [0,1], using the
            model's predicted mask quality.
          stability_score_thresh (float): A filtering threshold in [0,1], using
            the stability of the mask under changes to the cutoff used to binarize
            the model's mask predictions.
          stability_score_offset (float): The amount to shift the cutoff when
            calculated the stability score.
          box_nms_thresh (float): The box IoU cutoff used by non-maximal
            suppression to filter duplicate masks.
          crop_n_layers (int): If >0, mask prediction will be run again on
            crops of the image. Sets the number of layers to run, where each
            layer has 2**i_layer number of image crops.
          crop_nms_thresh (float): The box IoU cutoff used by non-maximal
            suppression to filter duplicate masks between different crops.
          crop_overlap_ratio (float): Sets the degree to which crops overlap.
            In the first crop layer, crops will overlap by this fraction of
            the image length. Later layers with more crops scale down this overlap.
          crop_n_points_downscale_factor (int): The number of points-per-side
            sampled in layer n is scaled down by crop_n_points_downscale_factor**n.
          point_grids (list(np.ndarray) or None): A list over explicit grids
            of points used for sampling, normalized to [0,1]. The nth grid in the
            list is used in the nth crop layer. Exclusive with points_per_side.
          min_mask_region_area (int): If >0, postprocessing will be applied
            to remove disconnected regions and holes in masks with area smaller
            than min_mask_region_area. Requires opencv.
          output_mode (str): The form masks are returned in. Can be 'binary_mask',
            'uncompressed_rle', or 'coco_rle'. 'coco_rle' requires pycocotools.
            For large resolutions, 'binary_mask' may consume large amounts of
            memory.
        """

'\nkwargs\n sam_kwargs (dict, optional): Optional arguments for fine-tuning the SAM model. Defaults to None.\n                The available arguments with default values are listed below. See https://bit.ly/410RV0v for more details.\n\n                points_per_side: Optional[int] = 32,\n                points_per_batch: int = 64,\n                pred_iou_thresh: float = 0.88,\n                stability_score_thresh: float = 0.95,\n                stability_score_offset: float = 1.0,\n                box_nms_thresh: float = 0.7,\n                crop_n_layers: int = 0,\n                crop_nms_thresh: float = 0.7,\n                crop_overlap_ratio: float = 512 / 1500,\n                crop_n_points_downscale_factor: int = 1,\n                point_grids: Optional[List[np.ndarray]] = None,\n                min_mask_region_area: int = 0,\n                output_mode: str = "binary_mask",\n\n        Using a SAM model, generates masks for the entire image.\n        Generates a grid of

In [8]:
# conversion function
def convert_to_8bit(file_path, out_path=None):
    "converts 32 bit float geotiffs to 8 bit for geosam"
    with rasterio.open(file_path) as src:
        img_32 = src.read()
        metadata = src.meta
        # convert
        img_8 = np.zeros((3, src.height, src.width), dtype=np.uint8)
        for band in range(3):
            band_data = img_32[band]
            img_8[band] = img_as_ubyte(band_data)

        metadata.update({'dtype': 'uint8',
                         'driver': 'GTiff',})

        if out_path is None:
            base, ext = os.path.split(file_path)
            out_path = f"{base}/8bit_{ext}"
         
        if os.path.exists(out_path):
            os.remove(out_path)

        
        with rasterio.open(out_path, 'w', **metadata) as dst:
           dst.write(img_8)
        
        return out_path

def convert_to_8bit_single(file_path, out_path=None):
    "converts 32 bit float geotiffs to 8 bit for geosam"
    with rasterio.open(file_path) as src:
        img_32 = src.read()
        metadata = src.meta
        # convert
        img_8 = np.zeros((3, src.height, src.width), dtype=np.uint8)
        for band in range(1):
            band_data = img_32[band]
            img_8[band] = img_as_ubyte(band_data)

        metadata.update({'dtype': 'uint8',
                         'driver': 'GTiff',})

        if out_path is None:
            base, ext = os.path.split(file_path)
            out_path = f"{base}/8bit_{ext}"
         
        if os.path.exists(out_path):
            os.remove(out_path)

        
        with rasterio.open(out_path, 'w', **metadata) as dst:
           dst.write(img_8)
        
        return out_path

In [6]:
# image selection function
def select_image(image):
    # display selected image
    m.layers[-1].visible = False
    m.add_raster(image, layer_name="Image")

    # set the image to segment
    sam.set_image(image)
    print(f"set image: {image}")


In [7]:

# set images
image1_32bit = '/bulk-1/rasters/2023-07-04_composite.tif'
image2_32bit = '/bulk-1/rasters/2023-07-11_composite.tif'
image3_32bit = '/bulk-1/rasters/2023-07-17_composite.tif'
image4_32bit = '/bulk-1/rasters/2023-07-24_composite.tif'
image5_32bit = '/bulk-1/rasters/2023-07-31_composite.tif'
image6_32bit = '/bulk-1/rasters/2023-08-08_composite.tif'
image7_32bit = '/bulk-1/rasters/2023-08-14_composite.tif'
image8_32bit = '/bulk-1/rasters/2023-08-22_composite.tif'
image9_32bit = '/bulk-1/rasters/2023-08-29_composite.tif'
image10_32bit = '/bulk-1/rasters/2023-09-05_composite.tif'


# convert images
image1 = convert_to_8bit(image1_32bit)
image2 = convert_to_8bit(image2_32bit)
image3 = convert_to_8bit(image3_32bit)
image4 = convert_to_8bit(image4_32bit)
image5 = convert_to_8bit(image5_32bit)
image6 = convert_to_8bit(image6_32bit)
image7 = convert_to_8bit(image7_32bit)
image8 = convert_to_8bit(image8_32bit)
image9 = convert_to_8bit(image9_32bit)
image10 = convert_to_8bit(image10_32bit)

In [10]:

# set images

image10_32bit = '/bulk-1/rasters/2023-09-05_index_nir.tif'



# convert images
image10 = convert_to_8bit_single(image10_32bit)

ValueError: Images of type float must be between -1 and 1.

In [14]:
select_image(image10)

TypeError: No such trait: LayerGroup.visible

In [15]:
m = sam.show_map()
m


Map(center=[49.8652955, -99.98090350000001], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_…

In [26]:
point_coords = [[-122.1419, 37.6383]]
sam.predict(point_coords, point_labels=1, point_crs="EPSG:32614", output="mask1.tif")
m.add_raster("mask1.tif", layer_name="Mask1", nodata=0, cmap="Blues", opacity=1)
m

point_coords = [[-122.1464, 37.6431], [-122.1449, 37.6415], [-122.1451, 37.6395]]
sam.predict(point_coords, point_labels=1, point_crs="EPSG:32614", output="mask2.tif")
m.add_raster("mask2.tif", layer_name="Mask2", nodata=0, cmap="Greens", opacity=1)
m

No valid pixel coordinates found.


IndexError: index 0 is out of bounds for axis 0 with size 0

In [38]:
# &&& add 5-counter polygons to map

In [ ]:
# &&& segment with scripted negative points
sam.predict(point_coords, point_labels=1, point_crs="EPSG:32614", output="mask1.tif")
point_labels, 1 fg 0 bg

In [ ]:
# &&& show points function, []to map
# &&& print current points function, map to []
# &&& show mask function
# &&& makseg function, args: maskName layerName points opt show points show mask

In [ ]:
# &&& check positions are correct in qgis

In [ ]:
makseg("seg_1", image10, points = [[-122.1419, 37.6383]], notpoints = [[-122.1419, 37.6383]], showp=True, showmk=True) 
# 2
# 3 not in field
# ...
# 1010